 # Lab 3 - 2D → 3D inference 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from itertools import combinations
import os

In [2]:

%matplotlib qt

In [3]:
#Read the image
os.getcwd()
img = Image.open('inference/images/rectangle.tif')

info_dict = {
    "Filename": img.filename,
    "Image Size": img.size,
    "Image Mode": img.mode,
    "Image Format": img.format,
    "Image width": img.width,
    "Image height": img.height,
    "Image is animated": getattr(img, 'is_animated'),
    "Frames in image": getattr(img, 'n_frames')
}
print(info_dict)

{'Filename': 'inference/images/rectangle.tif', 'Image Size': (640, 480), 'Image Mode': 'P', 'Image Format': 'TIFF', 'Image width': 640, 'Image height': 480, 'Image is animated': False, 'Frames in image': 1}


## Select 2 sets of parallel lines
To select 2 sets of parallel lines, execute the cell and click twice on the figure to create a line between two points. You can do this as many times for the first set of parallel lines you want to include. 

Once you are done with the first set, click once on the right mouse button to start selecting the lines for the second set. 

The two set of parallel lines should appear in red and blue. If you want to start again you can re-execute the cell. 

Once you are done, just go to thhe next cell (without reexecuting the current one)

In [37]:
x, y = [], [] # lists of the ends points of the first set of lines are stored
x2, y2 = [], [] # lists  of the ends points of the second set of lines
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(img.rotate(180), origin='lower', cmap='gray') # set the axis-origin to lower for better understandability (and rotate image)
first_set = True

def onclick(event):
    global first_set
    if event.button == 3: # right click
        first_set = False
    
    if event.button == 1: # mouse click for first set
        if first_set:
            x.append(float(event.xdata)) # store x and y for the clicked point
            y.append(float(event.ydata))
        else:
            x2.append(float(event.xdata))
            y2.append(float(event.ydata))
    for i in range(0, len(x), 2): # plot lines between points pairs
        ax.plot(x[i:i+2], y[i:i+2], 'ro-')
    for i in range(0, len(x2), 2): # plot lines between points pairs
        ax.plot(x2[i:i+2], y2[i:i+2], 'bo-')
    fig.canvas.draw()
    
cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()

## Calculate focal length f, the direction vectors w1 and w2, and the normal of the planar patch containing the rectangles

### You now have the x and y coordinates of the endpoints of the set of parallel lines you picked previously. You need the coordinates of the vanishing points for both directions.

In [46]:
# Coordinate of the vanishing points (find the intersection between the two set of lines). 
# You can use np.polyfit(x,y,1) which gives you the linear equation y = Ax + B that fit best a set of points
# and find y1 and y2 for the two set of lines
print("Obtained values:",x, y)
print("Obtained values2:",x2, y2)
print("Point1 of line 1", x[0], y[0])
print("Point2 of line 2", x[1], y[1])
print("Point1 of line 3", x[2], y[2])
print("Point2 of line 4", x[3], y[3])
print("Point1 of line 5", x2[0], y2[0])
print("Point2 of line 6", x2[1], y2[1])
print("Point1 of line 7", x2[2], y2[2])
print("Point2 of line 8", x2[3], y2[3])


Obtained values: [334.66842532467535, 509.9880275974027, 350.95292207792215, 566.6235795454547] [123.30884740259742, 244.28591720779227, 87.68587662337663, 242.52049512987017]
Obtained values2: [98.95413961038963, 321.0827922077923, 458.3088474025975, 239.24431818181822] [258.50568181818187, 155.33502435064938, 245.91943993506496, 332.5316558441559]
Point1 of line 1 334.66842532467535 123.30884740259742
Point2 of line 2 509.9880275974027 244.28591720779227
Point1 of line 3 350.95292207792215 87.68587662337663
Point2 of line 4 566.6235795454547 242.52049512987017
Point1 of line 5 98.95413961038963 258.50568181818187
Point2 of line 6 321.0827922077923 155.33502435064938
Point1 of line 7 458.3088474025975 245.91943993506496
Point2 of line 8 239.24431818181822 332.5316558441559


In [52]:
A1 = x.copy()  # first set x-values
B1 = y.copy()  # first set y-values

A2 = x2.copy() # second set x-values
B2 = y2.copy() # second set y-values

# --- Unpack points for set 1 (first 2 points for line 1, next 2 for line 2) ---
x1_1, y1_1 = A1[0], B1[0]
x1_2, y1_2 = A1[1], B1[1]
x1_3, y1_3 = A1[2], B1[2]
x1_4, y1_4 = A1[3], B1[3]

# --- Unpack points for set 2 ---
x2_1, y2_1 = A2[0], B2[0]
x2_2, y2_2 = A2[1], B2[1]
x2_3, y2_3 = A2[2], B2[2]
x2_4, y2_4 = A2[3], B2[3]

# --- Fit each line using np.polyfit ---
# Set 1:
m1, b1 = np.polyfit([x1_1, x1_2], [y1_1, y1_2], 1)  # Best-fit for line 1 (set 1)
m2, b2 = np.polyfit([x1_3, x1_4], [y1_3, y1_4], 1)  # Best-fit for line 2 (set 1)

# Set 2:
m3, b3 = np.polyfit([x2_1, x2_2], [y2_1, y2_2], 1)  # Best-fit for line 1 (set 2)
m4, b4 = np.polyfit([x2_3, x2_4], [y2_3, y2_4], 1)  # Best-fit for line 2 (set 2)

# --- Find intersection (vanishing point) for each set ---
# For set 1:
x_int1 = (b2 - b1) / (m1 - m2)
y_int1 = m1 * x_int1 + b1
vanish_set1 = (float(x_int1), float(y_int1))
print("Vanishing point for set 1:", vanish_set1)

# For set 2:
x_int2 = (b4 - b3) / (m3 - m4)
y_int2 = m3 * x_int2 + b3
vanish_set2 = (float(x_int2), float(y_int2))
print("Vanishing point for set 2:", vanish_set2)

# --- Plot the original image and overlay the extended lines ---
fig, ax = plt.subplots(figsize=(8, 8))
# Display the original image (rotated if needed)
ax.imshow(img.rotate(180), origin='lower', cmap='gray')

# For set 1: plot both lines as solid red lines extended to the vanishing point
ax.plot([x1_1, x1_2, x_int1], [y1_1, y1_2, y_int1], 'ro-', label='Set 1: Line 1')
ax.plot([x1_3, x1_4, x_int1], [y1_3, y1_4, y_int1], 'ro-', label='Set 1: Line 2')

# For set 2: plot both lines as solid blue lines extended to the vanishing point
ax.plot([x2_1, x2_2, x_int2], [y2_1, y2_2, y_int2], 'bo-', label='Set 2: Line 1')
ax.plot([x2_3, x2_4, x_int2], [y2_3, y2_4, y_int2], 'bo-', label='Set 2: Line 2')

# Mark the vanishing points with black circles
ax.plot(x_int1, y_int1, 'ko', markersize=8, label='Vanishing Pt Set 1')
ax.plot(x_int2, y_int2, 'ko', markersize=8, label='Vanishing Pt Set 2')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Extended Lines on Original Image')
ax.legend()
plt.show()


Vanishing point for set 1: (2031.4672735394222, 1294.1633898771217)
Vanishing point for set 2: (-1775.2979960495604, 1129.027439524215)


In [ ]:
u1 = x_int1
v1 = y_int1
u2 = x_int2
v2 = y_int2
# Assume the principal point is the center of the image.
# For example, if the image size is (width, height) from your info_dict:
u0 = img.width / 2.0
v0 = img.height / 2.0

print("Principal point: (", u0, ",", v0, ")")

# --- Compute focal length ---
term = (u1 - u0) * (u2 - u0) + (v1 - v0) * (v2 - v0)
f_squared = -term
if f_squared <= 0:
    raise ValueError("Computed f^2 is non-positive.")
f = np.sqrt(f_squared)
print("Focal length f:", f)

# --- Compute direction vectors ---
def compute_direction(u, v, u0, v0, f):
    vec = np.array([u - u0, v - v0, f], dtype=float)
    return vec / np.linalg.norm(vec)

w1 = compute_direction(u1, v1, u0, v0, f)
w2 = compute_direction(u2, v2, u0, v0, f)
print("Direction vector for set 1 (w1):", w1)
print("Direction vector for set 2 (w2):", w2)

# --- Compute the normal vector of the plane ---
normal = np.cross(w1, w2)
normal = normal / np.linalg.norm(normal)
print("Normal vector of the plane:", normal)

Principal point: ( 320.0 , 240.0 )
Focal length f: 1627.5299595426598
Direction vector for set 1 (w1): [0.66172934 0.40758643 0.62927544]
Direction vector for set 2 (w2): [-0.74882246  0.31772269  0.58165043]
Normal vector of the plane: [ 0.03713774 -0.85611074  0.5154563 ]


In [59]:
print("||w1|| =", np.linalg.norm(w1))
print("||w2|| =", np.linalg.norm(w2))
print("||normal|| =", np.linalg.norm(normal))

print("w1 dot w2 =", np.dot(w1, w2))
print("normal dot w1 =", np.dot(normal, w1))
print("normal dot w2 =", np.dot(normal, w2))



||w1|| = 1.0
||w2|| = 0.9999999999999999
||normal|| = 1.0
w1 dot w2 = 6.025064267227851e-17
normal dot w1 = 1.766058952834781e-17
normal dot w2 = -4.8362889280133765e-18


In [15]:
# The original parline function [TO BE MODIFIED !!!]
# I guess rewriting is having a look anyway
def parlines(lines):
    n = 3 # get 3 dimensions
    f = 1.0 # temporary value
    
    a = np.zeros((lines.shape[0], n)) 
    a[:,0] = lines[:,3] - lines[:,1]
    a[:,1] = -1*(lines[:,2] - lines[:,0])
    a[:,2] = np.multiply(lines[:,1], -1*a[:,1]) - np.multiply(lines[:,0],a[:,0])
    
    u, s, v = np.linalg.svd(a) 
    
    if (v[2] < 0).all(): # s and v are already sorted from largest to smallest
        v_min = np.abs(v[2])
    else:
        v_min = v[2]
        
    wvec = v_min.T
    wvec[0:2] /= f
    wvec /= np.linalg.norm(wvec)

    return wvec

In [ ]:
#  Rewritting the x and y coordinates to be read by the parlines function
lines = np.array([x, y]).T.reshape((len(x) // 2, 4)) # reshape to ((x,y), (x,y)) per line
lines2 = np.array([x2, y2]).T.reshape((len(x2) //2 , 4))

In [ ]:
# Calculate w1, w2

In [ ]:
# Calculate the normal plane